In [341]:
!pip install transformers

In [342]:
import os

import random
import numpy as np

import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import (
    AdamW,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_cosine_schedule_with_warmup,
)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import gc

from transformers import pipeline
import re

In [343]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()

# Load the training data
data = pd.read_csv('/kaggle/input/murderdata/Cain_s_Jawbone_Test_data.csv')


In [344]:
data

,OriginalPage,Text
0,1,I sit down alone at the appointed table and ta...
1,2,I plunged for the last time. The few remaining...
2,3,"At my meeting with Clement yesterday, he had\n..."
3,4,And I really think I would have preferred the\...
4,5,I hated my eye for being caught by what didn�t...
...,...,...
70,71,But next day that religious fellow�s head\ndra...
71,72,Of course I was sorry to say good-bye to old\n...
72,73,"All the artist in me flared up. After all, my\..."
73,74,"photographs of young and laughing athletes,\nl..."


In [345]:
split_data = pd.DataFrame(columns=['text', 'OriginalPage'])

In [346]:
for _, row in data.iterrows():
    # Split the text into two equal parts
    text1, text2 = row['Text'][:len(row['Text']) // 2], row['Text'][len(row['Text']) // 2:]
    # Add the split texts to the dataframe as separate rows
    split_data = split_data.append({'text': text1, 'OriginalPage': row['OriginalPage']}, ignore_index=True)
    split_data = split_data.append({'text': text2, 'OriginalPage': row['OriginalPage']}, ignore_index=True)
    
split_data=split_data.sample(frac=1)
# Print the resulting dataframe
split_data

,text,OriginalPage
23,"same\nname as the man who sang : �Ah, are you\...",12
26,"My ears were becoming attuned, and for the\nfi...",14
73,glanced at the\nrococo mirror on my left. Wel...,37
137,"self, ought I to\ndo? The answer was plain eno...",69
0,I sit down alone at the appointed table and ta...,1
...,...,...
88,"It was when that half Pole, half Frenchman,\na...",45
102,The cardinal was acquitted to-day of all\ncomp...,52
28,"Now, I considered, in my dear Lyons it would\n...",15
87,fruit merchant laying down his guitar and\nwip...,44


In [347]:
def remove_newlines(text):
    return re.sub(r'\n', '', text)

In [348]:
# Preprocess the text data by lowercasing and removing punctuation
split_data['text'] = split_data['text'].apply(remove_newlines)

In [349]:
X = split_data['text']
X = X.str.lower().str.replace(r'[^\w\s]', '')
y = split_data['OriginalPage']

In [350]:
vectorizer = CountVectorizer()
tfidf_data = vectorizer.fit_transform(X).toarray()
features = vectorizer.get_feature_names_out()
tfidf_data = pd.DataFrame(tfidf_data, columns=features)

In [351]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest=train_test_split(tfidf_data, y, test_size=0.1, random_state=11)

In [352]:
# !pip install lazypredict

In [388]:
from lazypredict.Supervised import LazyRegressor
from sklearn.utils import shuffle
import numpy as np

reg = LazyRegressor(verbose=0,ignore_warnings=False, custom_metric=None )
models,predictions = reg.fit(xtrain, xtest, ytrain, ytest)

NameError: name 'accuracy' is not defined

In [354]:
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
Lars,222378847226601786997728763445248.00,-61710130105381995315408979552632832.00,5952619796296872960.00,0.82
SGDRegressor,930525450920781056.00,-258220812630516727808.00,385057454108.33,0.20
OrthogonalMatchingPursuit,1.01,-3.00,47.90,0.22
KernelRidge,1.01,-1.97,41.32,0.24
GaussianProcessRegressor,1.01,-1.93,41.00,0.23
DecisionTreeRegressor,1.01,-1.71,39.47,0.21
ExtraTreeRegressor,1.01,-1.57,38.43,0.21
MLPRegressor,1.01,-0.51,29.46,3.67
OrthogonalMatchingPursuitCV,1.01,-0.44,28.75,0.80


In [411]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import OrthogonalMatchingPursuitCV
from sklearn.linear_model import QuantileRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier , StackingRegressor, VotingRegressor
from sklearn.linear_model import OrthogonalMatchingPursuitCV, ElasticNetCV



model1 = OrthogonalMatchingPursuitCV()
model2 = ElasticNetCV()
model3 = QuantileRegressor()
voting_model = VotingRegressor(estimators=[('model_1', model1), ('model_2', model2)
                                            , ('model_3', model3)])


# Make predictions on the test data

reg = voting_model.fit(tfidf_data, y)

In [356]:
# from sklearn.linear_model import OrthogonalMatchingPursuitCV
# reg = OrthogonalMatchingPursuitCV(cv=5).fit(tfidf_data, y)

In [357]:
# Load the test data
test_data = pd.read_csv('/kaggle/input/murderdata/Cain_s_Jawbone_Test_data.csv')
test_data['Text'] = test_data['Text'].apply(remove_newlines)
X_test = test_data['Text']
# Preprocess the test data in the same way as the training data
X_test = X_test.str.lower().str.replace(r'[^\w\s]', '')
# Extract features from the test data using the same vectorizer


In [358]:
testt = vectorizer.transform(X_test).toarray()
features = vectorizer.get_feature_names_out()
test_data = pd.DataFrame(testt, columns=features)

In [412]:
y__pred=reg.predict(test_data)

In [413]:
y__pred

array([38.26712329, 28.25      , 38.26712329, 38.26712329, 38.26712329,
       38.26712329, 28.25      , 38.26712329, 38.26712329, 28.25      ,
       38.26712329, 38.26712329, 38.26712329, 38.26712329, 38.26712329,
       28.25      , 38.26712329, 38.26712329, 38.26712329, 38.26712329,
       38.26712329, 38.26712329, 38.26712329, 38.26712329, 38.26712329,
       38.26712329, 38.26712329, 38.26712329, 38.26712329, 38.26712329,
       38.26712329, 38.26712329, 38.26712329, 38.26712329, 38.26712329,
       38.26712329, 38.26712329, 38.26712329, 38.26712329, 38.26712329,
       38.26712329, 38.26712329, 38.26712329, 38.26712329, 38.26712329,
       38.26712329, 38.26712329, 38.26712329, 38.26712329, 38.26712329,
       38.26712329, 38.26712329, 38.26712329, 38.26712329, 38.26712329,
       38.26712329, 38.26712329, 38.26712329, 38.26712329, 38.26712329,
       38.26712329, 38.26712329, 38.26712329, 38.26712329, 38.26712329,
       38.26712329, 38.26712329, 38.26712329, 38.26712329, 38.26

In [414]:
pred=[round((i)) for i in y__pred]

In [404]:
pred=[round((i)) for i in y_pred]

In [383]:
sub=pd.read_csv("/kaggle/input/murderdata/SampleSubmission.csv")

In [415]:
sub["CorrectPage"] = pred

In [416]:
sub.CorrectPage

0     38
1     28
2     38
3     38
4     38
      ..
70    38
71    38
72    38
73    38
74    38
Name: CorrectPage, Length: 75, dtype: int64

In [417]:
sub.CorrectPage.nunique()

2

In [418]:
sub.to_csv("sub11.csv", index=False)

In [419]:
sub

,OriginalPage,CorrectPage
0,1,38
1,2,28
2,3,38
3,4,38
4,5,38
...,...,...
70,71,38
71,72,38
72,73,38
73,74,38
